In [3]:
# 7월 16일 13:16 학습 + 장르 가중치 적용
import json
from konlpy.tag import Okt
from gensim.models import Word2Vec
import numpy as np
import joblib

# 경로 설정
DB_PATH = "C:/kosmo/data/"
PATH = "C:/kosmo/test_data/IH/"
file_name = "movies_DB_partial.json"

# 영화 데이터 불러오기
with open(DB_PATH + file_name, encoding="utf-8") as f:
    movies = json.load(f)

# 형태소 분석기
okt = Okt()

# 제목 + 줄거리 + 장르 토큰 추출 함수 (장르 가중치 반영)
def get_tokens(title, overview, genres, genre_weight=3):
    title = title or ""
    overview = overview or ""
    genres = genres or []

    tokens = []

    # 제목 토큰
    for word, pos in okt.pos(title, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            tokens.append(word)

    # 줄거리 토큰
    for word, pos in okt.pos(overview, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            tokens.append(word)

    # 장르 토큰 (가중치 부여)
    tokens.extend(genres * genre_weight)

    return tokens

# 전체 영화에 대해 토큰화
keyword = []
for movie in movies:
    tokens = get_tokens(movie.get('title_ko'), movie.get('overview'), movie.get('genres'), genre_weight=3)
    keyword.append(tokens)

# Word2Vec 학습
model = Word2Vec(
    keyword,
    vector_size=100,
    window=5,
    min_count=1,  # 등장 빈도 1 이상 단어 포함
    workers=4,
    sg=0  # CBOW 방식
)

# 제목 토큰화 (검색용, overview/genre는 제외)
title_tokens = [
    get_tokens(movie.get('title_ko'), "", [], genre_weight=3)
    for movie in movies
]

joblib.dump(title_tokens, PATH + "title_tokens.pkl")

# 영화 벡터 생성 함수 (토큰 평균)
def get_movie_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    return np.mean(np.array(vectors), axis=0) if vectors else np.zeros(model.vector_size)

# 영화 벡터 계산
movie_vectors = [get_movie_vector(tokens, model) for tokens in keyword]

# 모델 및 데이터 저장
joblib.dump(model, PATH + "word2vec_movie.model")
joblib.dump(movie_vectors, PATH + "movie_vectors.pkl")
joblib.dump(movies, PATH + "movies.pkl")
joblib.dump(keyword, PATH + "keyword.pkl")

print("✅ 장르 가중치 적용하여 학습 및 저장 완료")

✅ 장르 가중치 적용하여 학습 및 저장 완료


In [10]:
import sys
import json
import numpy as np
import joblib
from soynlp.tokenizer import LTokenizer  # soynlp의 단순 토크나이저 사용 (공백 기준)

# --- 파일 경로 설정 ---
PATH = "C:/kosmo/model/"  # 모델 및 데이터 저장 경로

# --- 모델 및 데이터 불러오기 ---
model = joblib.load(PATH + "word2vec_movie.model")        # 학습된 Word2Vec 모델
movie_vectors = np.array(joblib.load(PATH + "movie_vectors.pkl"))  # 영화별 벡터 (줄거리+장르 기반)
movies = joblib.load(PATH + "movies.pkl")                 # 영화 정보 리스트 (제목, 줄거리, 장르 등)
title_tokens = joblib.load(PATH + "title_tokens.pkl")     # 영화 제목을 토큰화한 리스트 (검색용)

# --- 제목 검색용 토크나이저 준비 (공백 기준 LTokenizer 사용) ---
tokenizer = LTokenizer()

def tokenize_title(text):
    """입력 제목에서 불필요한 공백 제거 후 토큰화"""
    if not text:
        return []
    text = text.strip().replace(" ", "")  # ← 공백 제거
    return tokenizer.tokenize(text)

def jaccard_similarity(set1, set2):
    """두 집합 간 자카드 유사도 계산"""
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

def find_similar_movie(input_title, title_tokens, threshold=0.1):
    """
    입력한 제목과 가장 유사한 제목을 가진 영화의 인덱스를 반환.
    - 유사도는 자카드 유사도 기반
    - 유사도가 threshold 미만이면 -1 반환
    """
    input_tokens = set(tokenize_title(input_title))
    best_match_idx = -1
    best_score = 0.0
    for idx, tokens in enumerate(title_tokens):
        score = jaccard_similarity(input_tokens, set(tokens))
        if score > best_score and score >= threshold:
            best_score = score
            best_match_idx = idx
    return best_match_idx

def recommend_movie(input_title, movies, movie_vectors, title_tokens, top_n=5):
    """
    입력한 제목과 유사한 영화 1개를 찾은 후,
    그 영화의 벡터(장르+줄거리)를 기준으로 가장 유사한 영화 top_n개를 추천
    """
    matched_idx = find_similar_movie(input_title, title_tokens)

    # 유사한 제목을 찾지 못했을 경우: 모든 영화 벡터의 평균을 사용
    if matched_idx == -1:
        print(f"'{input_title}'과(와) 비슷한 영화를 찾지 못했습니다. 전체 평균 벡터로 추천합니다.")
        input_vec = np.mean(movie_vectors, axis=0)
    else:
        print(f"입력하신 '{input_title}'과(와) 가장 비슷한 영화는 '{movies[matched_idx]['title_ko']}'입니다.")
        input_vec = movie_vectors[matched_idx]

    # --- 코사인 유사도 계산 ---
    input_norm = np.linalg.norm(input_vec)
    vectors_norm = np.linalg.norm(movie_vectors, axis=1)
    dot_products = movie_vectors @ input_vec  # 내적

    with np.errstate(divide='ignore', invalid='ignore'):
        cosine_similarities = dot_products / (vectors_norm * input_norm)  # 유사도 계산
        cosine_similarities = np.nan_to_num(cosine_similarities)  # NaN 방지

    # 자기 자신은 제외 (중복 추천 방지)
    if matched_idx >= 0:
        cosine_similarities[matched_idx] = -1

    # 상위 top_n 유사한 영화 인덱스 선택
    top_indices = np.argpartition(-cosine_similarities, top_n)[:top_n]
    top_indices = top_indices[np.argsort(-cosine_similarities[top_indices])]  # 유사도 내림차순 정렬

    # 추천 영화 정보 구성
    recommendations = []
    for i in top_indices:
        m = movies[i]
        recommendations.append({
            "title_ko": m.get('title_ko', ''),
            "release_date": m.get('release_date', ''),
            "vote_average": m.get('vote_average', 0),
            "poster_path": m.get('poster_path', ''),
            "similarity": cosine_similarities[i]
        })

    return recommendations

if __name__ == "__main__":
    # 예시: 사용자 입력 영화 제목
    input_title = "인 셉 션"  # ← 원하는 제목으로 변경 가능

    # 추천 실행
    recs = recommend_movie(input_title, movies, movie_vectors, title_tokens, top_n=5)

    # 추천 결과 출력
    print(f"\n✅ '{input_title}' 추천 영화 리스트:")
    for rec in recs:
        print(f"🎬 {rec['title_ko']} (개봉일: {rec['release_date']}, 평점: {rec['vote_average']:.1f})")
        print(f"포스터: {rec['poster_path']}")
        print(f"유사도: {rec['similarity']:.3f}\n")

입력하신 '인 셉 션'과(와) 가장 비슷한 영화는 '인셉션'입니다.

✅ '인 셉 션' 추천 영화 리스트:
🎬 에반게리온: Q (개봉일: 2012-11-17, 평점: 7.2)
포스터: https://image.tmdb.org/t/p/w500/zERMhjBDiYoaHGDRsKP6IxzuV5F.jpg
유사도: 0.948

🎬 스워드피쉬 (개봉일: 2001-06-08, 평점: 6.3)
포스터: https://image.tmdb.org/t/p/w500/63YY0GuORn1G73SAIADVDATdF5A.jpg
유사도: 0.947

🎬 서복 (개봉일: 2021-04-12, 평점: 7.2)
포스터: https://image.tmdb.org/t/p/w500/6R3eKsdVS7ZWrY28QUt3DCG3N72.jpg
유사도: 0.944

🎬 파이프라인 (개봉일: 2021-05-26, 평점: 7.0)
포스터: https://image.tmdb.org/t/p/w500/oo4sBEhrOovS1l1qcPo0i1RRacP.jpg
유사도: 0.943

🎬 두뇌유희프로젝트, 퍼즐 (개봉일: 2006-09-14, 평점: 5.2)
포스터: https://image.tmdb.org/t/p/w500/ts7LKneqQ9MCOwCxd0e1SzQxtX2.jpg
유사도: 0.942

